In [1]:
import numpy as np
import cv2 as cv
import matplotlib.pyplot as plt
import math

In [2]:
frame = cv.Mat
paused = False
cap = cv.VideoCapture('../src/video.mp4')
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))
size = (frame_width, frame_height)
times = []  
while cap.isOpened():
    if not paused:
        ret, frame = cap.read()
        if not ret:
            print("Can't receive frame (stream end?). Exiting ...")
            break

    frame = cv.resize(frame, (640, 480))
    frame = cv.cvtColor(frame,cv.COLOR_BGR2GRAY)

    # calculate shifted dft
    dft = cv.dft(np.float32(frame),flags = cv.DFT_COMPLEX_OUTPUT)
    dft_shift = np.fft.fftshift(dft)
    real = dft_shift[:,:,0]
    imag = dft_shift[:,:,1]

    # calculate spectrum
    spectrum = 20*np.log(cv.magnitude(real,imag)+0.0001)
    spectrum_norm = spectrum/np.max(spectrum)
    
    cv.imshow('spectrum', spectrum_norm)
    cv.imshow('frame', frame)

    key = cv.waitKey(30)
    if key == 27:  # Esc key to exit
        break
    elif key == ord('p'):  # Pause
        paused = True
    elif key == ord('o'):  # Continue
        paused = False

cap.release()
cv.destroyAllWindows()

qt.qpa.plugin: Could not find the Qt platform plugin "wayland" in "/home/kaio/.local/lib/python3.10/site-packages/cv2/qt/plugins"


# Questão 1

In [3]:
# Função para criar a máscara do filtro passa-baixa ideal
def ideal_low_pass_filter(shape, D0):
    rows, cols = shape
    center = (rows // 2, cols // 2)
    mask = np.zeros((rows, cols, 2), np.float32)
    for i in range(rows):
        for j in range(cols):
            D = np.sqrt((i - center[0]) ** 2 + (j - center[1]) ** 2)
            if D <= D0:
                mask[i, j] = 1  # Passa-baixa ideal
    return mask

In [4]:
# Parâmetros do vídeo e inicialização
paused = False
cap = cv.VideoCapture('../src/video.mp4')
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))

# Valor de corte D0 para o filtro passa-baixa
D0 = 50  # Altere este valor para testar diferentes frequências de corte

while cap.isOpened():
    if not paused:
        ret, frame = cap.read()
        if not ret:
            print("Can't receive frame (stream end?). Exiting ...")
            break

    # Redimensiona o frame e converte para escala de cinza
    frame = cv.resize(frame, (640, 480))
    gray_frame = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)

    # Calcula a DFT e aplica o deslocamento de frequências
    dft = cv.dft(np.float32(gray_frame), flags=cv.DFT_COMPLEX_OUTPUT)
    dft_shift = np.fft.fftshift(dft)

    # Cria e aplica o filtro passa-baixa ideal
    mask = ideal_low_pass_filter(gray_frame.shape, D0)
    dft_filtered = dft_shift * mask

    # Aplica o deslocamento inverso e calcula a IDFT
    dft_ishift = np.fft.ifftshift(dft_filtered)
    img_back = cv.idft(dft_ishift)
    img_back = cv.magnitude(img_back[:, :, 0], img_back[:, :, 1])

    # Normaliza o espectro para visualização
    real = dft_shift[:, :, 0]
    imag = dft_shift[:, :, 1]
    spectrum = 20 * np.log(cv.magnitude(real, imag) + 0.0001)
    spectrum_norm = spectrum / np.max(spectrum)

    # Exibe o espectro e o vídeo filtrado
    cv.imshow('Spectrum', spectrum_norm)
    cv.imshow('Filtered Video', img_back / np.max(img_back))
    cv.imshow('Original Video', gray_frame)

    # Controle de pausa e saída
    key = cv.waitKey(30)
    if key == 27:  # Esc para sair
        break
    elif key == ord('p'):  # Pausar
        paused = True
    elif key == ord('o'):  # Continuar
        paused = False
    elif key == ord('d'):  # Alterar o valor de D0 dinamicamente
        D0 += 10  # Aumentar o raio de corte
    elif key == ord('a'):
        D0 = max(10, D0 - 10)  # Diminuir o raio de corte, mas manter positivo

# Libera o vídeo e destrói as janelas
cap.release()
cv.destroyAllWindows()

# Questão 2

In [5]:
# Função para criar a máscara do filtro passa-baixa de Butterworth
def butterworth_low_pass_filter(shape, D0, n):
    rows, cols = shape
    center = (rows // 2, cols // 2)
    mask = np.zeros((rows, cols, 2), np.float32)
    
    for i in range(rows):
        for j in range(cols):
            D = np.sqrt((i - center[0]) ** 2 + (j - center[1]) ** 2)
            mask_value = 1 / (1 + (D / D0) ** (2 * n))  # Fórmula do filtro de Butterworth
            mask[i, j] = mask_value
    
    return mask

In [6]:
# Parâmetros do vídeo e inicialização
paused = False
cap = cv.VideoCapture('../src/video.mp4')
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))

# Valores iniciais de D0 e n para o filtro Butterworth
D0 = 50  # Frequência de corte inicial
n = 2    # Ordem do filtro inicial

while cap.isOpened():
    if not paused:
        ret, frame = cap.read()
        if not ret:
            print("Can't receive frame (stream end?). Exiting ...")
            break

    # Redimensiona o frame e converte para escala de cinza
    frame = cv.resize(frame, (640, 480))
    gray_frame = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)

    # Calcula a DFT e aplica o deslocamento de frequências
    dft = cv.dft(np.float32(gray_frame), flags=cv.DFT_COMPLEX_OUTPUT)
    dft_shift = np.fft.fftshift(dft)

    # Cria e aplica o filtro passa-baixa de Butterworth
    mask = butterworth_low_pass_filter(gray_frame.shape, D0, n)
    dft_filtered = dft_shift * mask

    # Aplica o deslocamento inverso e calcula a IDFT
    dft_ishift = np.fft.ifftshift(dft_filtered)
    img_back = cv.idft(dft_ishift)
    img_back = cv.magnitude(img_back[:, :, 0], img_back[:, :, 1])

    # Normaliza o espectro para visualização
    real = dft_shift[:, :, 0]
    imag = dft_shift[:, :, 1]
    spectrum = 20 * np.log(cv.magnitude(real, imag) + 0.0001)
    spectrum_norm = spectrum / np.max(spectrum)

    # Exibe o espectro e o vídeo filtrado
    cv.imshow('Spectrum', spectrum_norm)
    cv.imshow('Filtered Video', img_back / np.max(img_back))
    cv.imshow('Original Video', gray_frame)

    # Controle de pausa e saída
    key = cv.waitKey(30)
    if key == 27:  # Esc para sair
        break
    elif key == ord('p'):  # Pausar
        paused = True
    elif key == ord('o'):  # Continuar
        paused = False
    elif key == ord('d'):  # Aumentar D0
        D0 += 10
    elif key == ord('a'):  # Diminuir D0
        D0 = max(10, D0 - 10)
    elif key == ord('n'):  # Aumentar ordem do filtro
        n += 1
    elif key == ord('m'):  # Diminuir ordem do filtro
        n = max(1, n - 1)

# Libera o vídeo e destrói as janelas
cap.release()
cv.destroyAllWindows()

# Questão 3

In [7]:
# Função para criar a máscara do filtro passa-alta gaussiano
def gaussian_high_pass_filter(shape, D0):
    rows, cols = shape
    center = (rows // 2, cols // 2)
    mask = np.zeros((rows, cols, 2), np.float32)
    
    for i in range(rows):
        for j in range(cols):
            D = np.sqrt((i - center[0]) ** 2 + (j - center[1]) ** 2)
            mask_value = 1 - np.exp(-(D ** 2) / (2 * (D0 ** 2)))  # Fórmula do passa-alta gaussiano
            mask[i, j] = mask_value
    
    return mask

In [8]:
# Parâmetros do vídeo e inicialização
paused = False
cap = cv.VideoCapture('../src/video.mp4')
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))

# Valor inicial de D0 para o filtro passa-alta gaussiano
D0 = 50  # Frequência de corte inicial

while cap.isOpened():
    if not paused:
        ret, frame = cap.read()
        if not ret:
            print("Can't receive frame (stream end?). Exiting ...")
            break

    # Redimensiona o frame e converte para escala de cinza
    frame = cv.resize(frame, (640, 480))
    gray_frame = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)

    # Calcula a DFT e aplica o deslocamento de frequências
    dft = cv.dft(np.float32(gray_frame), flags=cv.DFT_COMPLEX_OUTPUT)
    dft_shift = np.fft.fftshift(dft)

    # Cria e aplica o filtro passa-alta gaussiano
    mask = gaussian_high_pass_filter(gray_frame.shape, D0)
    dft_filtered = dft_shift * mask

    # Aplica o deslocamento inverso e calcula a IDFT
    dft_ishift = np.fft.ifftshift(dft_filtered)
    img_back = cv.idft(dft_ishift)
    img_back = cv.magnitude(img_back[:, :, 0], img_back[:, :, 1])

    # Normaliza o espectro para visualização
    real = dft_shift[:, :, 0]
    imag = dft_shift[:, :, 1]
    spectrum = 20 * np.log(cv.magnitude(real, imag) + 0.0001)
    spectrum_norm = spectrum / np.max(spectrum)

    # Exibe o espectro e o vídeo filtrado
    cv.imshow('Spectrum', spectrum_norm)
    cv.imshow('Filtered Video', img_back / np.max(img_back))
    cv.imshow('Original Video', gray_frame)

    # Controle de pausa e saída
    key = cv.waitKey(30)
    if key == 27:  # Esc para sair
        break
    elif key == ord('p'):  # Pausar
        paused = True
    elif key == ord('o'):  # Continuar
        paused = False
    elif key == ord('d'):  # Aumentar D0
        D0 += 10
    elif key == ord('a'):  # Diminuir D0
        D0 = max(10, D0 - 10)

# Libera o vídeo e destrói as janelas
cap.release()
cv.destroyAllWindows()